In [2]:
import dask 
import numpy as np
import dask.dataframe as dd 
from collections import OrderedDict

# Load data

In [2]:
import os


os.environ["DASK_TEMPORARY_DIRECTORY"] = "smt/dask"
os.environ["PGEOCODE_DATA_DIR"] = "smt/pgeocode_data"
if not os.path.exists("smt"):
    os.mkdir("smt")
if not os.path.exists("smt/dask"):
    os.mkdir("smt/dask")
if not os.path.exists("smt/pgeocode_data"):
    os.mkdir("smt/pgeocode_data")
import pandas as pd
import hvplot.pandas
import dask
import hvplot.dask
import dask.dataframe as dd
import json
import tqdm
from scipy.spatial.distance import cdist
import seaborn as sns
import matplotlib.pyplot as plt
import scipy.stats as stats
import numpy as np
import statsmodels.api as sm
from statsmodels.formula.api import ols
import pingouin as pg
import pgeocode as geo
import geopandas as gpd
import fiona
import panel as pn
pn.extension(comms="vscode")
from scipy.stats import entropy


ModuleNotFoundError: No module named 'hvplot'

### Specification of the data folder 
options: 
- "preflight" for a sample 
- "reduced" for all data

In [3]:
DATA_FOLDER = "preflight"

### Load views data

In [4]:
ddf_views = dd.read_csv(f'data_dpg_testdata/{DATA_FOLDER}/reduced_views.csv', dtype={
       'GEO_ZIPCODE': object,
       'REFR_MEDIUM': object, 'IS_LOGGED_IN': object, 'file_name': object,
       "privacy_advertising":int,       
       "privacy_analytics":int,       
       "privacy_functional":int,       
       "privacy_geo_location":int,       
       "privacy_marketing":int,       
       "privacy_non-personalised_ads":int,       
       "privacy_personalisation":int,       
       "privacy_social_media":int,       
       "privacy_target_advertising":int,       
       'ARTICLE_ID':"str",
       })
ddf_views["article_id"] = ddf_views["ARTICLE_ID"]
# ddf_views = ddf_views.drop(["ARTICLE_ID"] , axis=1)
ddf_views["DERIVED_TSTAMP"] = dd.to_datetime(ddf_views["DERIVED_TSTAMP"])
ddf_views["dt_hour"] = ddf_views["DERIVED_TSTAMP"].dt.hour
ddf_views["dt_weekday"] = ddf_views["DERIVED_TSTAMP"].dt.weekday
ddf_views["dt_dayofmonth"] = ddf_views["DERIVED_TSTAMP"].dt.day
ddf_views["dt_month"] = ddf_views["DERIVED_TSTAMP"].dt.month
ddf_views = ddf_views.replace("nu.web.advertorial", "nu.web")
ddf_views_unidentified = ddf_views[ddf_views["QUASI_USER_ID"].isna()].copy()
ddf_views = ddf_views[~ddf_views["QUASI_USER_ID"].isna()]
device_counts = ddf_views["APP_ID"].value_counts().compute()
# sample_amount = (device_counts.min()//100000)*100000
sample_amount = device_counts.min()
# sample_amount = 1000000
unique_devices = device_counts.index.unique()
columns_privacy = list(ddf_views.columns[ddf_views.columns.str.startswith("privacy_")].values)
columns_time = ["hour", "weekday", "dayofmonth", "month"]
ddf_views = ddf_views.groupby('APP_ID').apply(lambda df: df.sample(sample_amount if len(df) else 0) , meta=ddf_views.partitions[0]).reset_index(drop=True)
ddf_views = ddf_views.set_index('article_id')
ddf_views

FileNotFoundError: [Errno 2] No such file or directory: '/home/chandni/Documents/UNI/DPGResearch/ClusterProfiles/code/data_dpg_testdata/preflight/reduced_views.csv'

In [5]:
# grab a sample
ddf_sample, rest = ddf_views.random_split([0.0001, 0.9999])

NameError: name 'ddf_views' is not defined

In [6]:
# select the columns you want to work with
views_small = ddf_sample.drop(["GEO_CITY", "GEO_REGION", "REFR_MEDIUM",'privacy_functional',
       'privacy_analytics', 'privacy_target_advertising',
       'privacy_personalisation', 'privacy_non-personalised_ads',
       'privacy_marketing', 'privacy_social_media', 'privacy_geo_location',
       'privacy_advertising', 'file_name'], axis = 1)

NameError: name 'ddf_sample' is not defined

### Get some numbers 

In [ ]:
# How many non-NL GEO-COUNTRY? 

In [12]:
country_counts = ddf_views["GEO_COUNTRY"].value_counts().compute()
print(country_counts)
print(type(country_counts))

NL    12356429
ES      118474
DE      117140
FR      106793
BE       98041
        ...   
PM           1
VA           1
BL           1
FK           1
IO           1
Name: GEO_COUNTRY, Length: 223, dtype: int64
<class 'pandas.core.series.Series'>


In [15]:
country_counts.to_csv("country_counts_preflight.csv")

In [ ]:
# How many unique QUASI_USER_ID? 

In [6]:
id_counts = ddf_views["QUASI_USER_ID"].value_counts().compute()
print(id_counts)

dee1f36084ab7b8be4a9d993982f807a    753
b20d4f85f00f19ca790009bc1924856d    545
0322f1afbeb4505a71afce2d9364d87b    267
b8f0755f048b8bc50de36833db1b9103    257
2c928756a86f21b276fbd3a04971c289    235
                                   ... 
8d8bb4bf4b397e01548719507d569ce0      1
8d8bb71ea4efd573b706716c12b259a7      1
8d8bbf635aec85bf9574b5a27d943ad7      1
8d8bbfa5b98adbd2cd567d9ed7a5b586      1
800b9f7730ea538011b44d01aad64ad1      1
Name: QUASI_USER_ID, Length: 4316046, dtype: int64


In [ ]:
## Interaction 

drop = ['GEO_REGION', 'REFR_URLHOST', 'privacy_functional',
       'privacy_analytics', 'privacy_target_advertising',
       'privacy_personalisation', 'privacy_non-personalised_ads',
       'privacy_marketing', 'privacy_social_media', 'privacy_geo_location',
       'privacy_advertising', 'file_name'] 




In [ ]:
drop = ["SE_CATEGORY","SE_VALUE","GEO_CITY", "GEO_REGION", "REFR_MEDIUM",'privacy_functional',
       'privacy_analytics', 'privacy_target_advertising',
       'privacy_personalisation', 'privacy_non-personalised_ads',
       'privacy_marketing', 'privacy_social_media', 'privacy_geo_location',
       'privacy_advertising', 'file_name']

### Load article data

In [7]:
DATA_FOLDER_ARTICLES = "reduced"
ddf_article = dd.read_csv(f'data_dpg_testdata/{DATA_FOLDER_ARTICLES}/reduced_articles_df_article.csv', dtype={'sub_section': 'object', 'url': 'object'})
ddf_article = ddf_article.set_index('article_id')
ddf_article.index = ddf_article.index.astype('str')
ddf_article.head()

,cds_content_id,brands,title,text,authors,url,main_section,sub_section,num_words,num_sentences,num_chars,first_publication_timestamp,categories_generated,keywords_curated,brand_safety_is_brandsafe,brand_safety_confidence,file_name
article_id,,,,,,,,,,,,,,,,,
537071,article-mm-6229c19da07c8a000163379b,nu.nl,Peugeot 307 in alle opzichten verbeterd,Het nieuwe gezicht komt op alle carrosserievar...,ANP,https://nu.nl/auto/537071/peugeot-307-in-alle-...,Economie,Onderweg,351,27,2327,NaN,Onderweg,NaN,True,0.1,3
962788,article-mm-621539e134c4a7000191db08,nu.nl,Hier is hij dan: de nieuwe Volvo V70!,De voorzijde van de Volvo V70 krijgt trekken v...,autoweek.nl,https://nu.nl/auto/962788/hier-is-hij-dan-de-n...,Onderweg,NaN,0,0,0,NaN,Onderweg,NaN,True,0.1,1
1013291,article-mm-6239e222e28ee500012c67ff,nu.nl,De Peugeot 207 SW!,In de Peugeot-vestiging in Poissy is de voorse...,autoweek.nl,https://nu.nl/auto/1013291/de-peugeot-207-sw.html,Onderweg,NaN,170,17,1081,NaN,Onderweg,NaN,True,0.1,2
1149251,article-mm-6229c559c1fdac00016a6b3e,nu.nl,Meer details nieuwe Opel Agila,De beelden die we eerder al liet zien toonden ...,autoweek.nl,https://nu.nl/auto/1149251/meer-details-nieuwe...,Onderweg,NaN,361,27,2134,NaN,Onderweg,NaN,True,0.1,3
1401797,article-mm-626a799867ff2a00019c059e,nu.nl,Oranje meest populair voor Dodge Charger,Dodge heeft onlangs de Charger nieuw leven ing...,autoweek.nl,https://nu.nl/auto/1401797/oranje-meest-popula...,Economie,Onderweg,178,20,1133,NaN,Onderweg,NaN,True,0.1,3


In [8]:
ddf_topics = dd.read_csv(f'data_dpg_testdata/{DATA_FOLDER_ARTICLES}/reduced_articles_df_fixed_set_topics.csv', dtype={"article_id":"str"})
topic_columns = ddf_topics.columns[ddf_topics.columns.str.startswith("topic_")].values
ddf_topics = ddf_topics.reset_index().set_index('index')
ddf_topics = ddf_topics.assign(article_topic_entropy = dd.from_array(entropy(ddf_topics[topic_columns], axis=1)))
ddf_topics = ddf_topics.assign(argmax_topic = dd.from_array(ddf_topics[topic_columns].idxmax(axis=1).compute().values))
ddf_topics = ddf_topics.set_index('article_id')
ddf_topics.head()

,topic_voetbal,topic_optredens_en_voorstellingen,topic_politiek,topic_technologie_en_handel,topic_relatie_en_opvoeden,topic_bestuur_en_organisatie,topic_human_interest,topic_verkeersongevallen,topic_nominatie_en_prijzen,topic_veiligheid,...,topic_overheidsbeleid,topic_vakbond_club_en_vereniging,topic_religie,topic_bouwen_en_wonen,topic_recht_en_justitie,topic_samenleving,topic_woon_en_leefomgeving,file_name,article_topic_entropy,argmax_topic
article_id,,,,,,,,,,,,,,,,,,,,,
1013291,0.105175,0.028085,0.001613,0.025450,0.024846,0.001953,0.036237,0.063389,0.041297,0.001265,...,0.024950,0.049110,0.001384,0.001251,0.001558,0.001142,0.054470,2,2.660889,topic_duurzaamheid
1149251,0.096021,0.001019,0.000872,0.021542,0.002109,0.001056,0.011584,0.148419,0.000740,0.000684,...,0.001159,0.014120,0.000748,0.000677,0.000842,0.000617,0.039665,3,2.116882,topic_duurzaamheid
1401797,0.002146,0.001896,0.001623,0.002359,0.004391,0.001965,0.002967,0.001080,0.022382,0.001273,...,0.002157,0.019468,0.001393,0.001259,0.001567,0.001149,0.003774,3,2.471787,topic_kunst_en_cultuur
1420717,0.001877,0.032115,0.001420,0.044774,0.003638,0.063106,0.002595,0.068904,0.001204,0.001114,...,0.001887,0.155525,0.001218,0.001101,0.001371,0.001005,0.022787,3,2.527454,topic_vakbond_club_en_vereniging
1505884,0.002216,0.001958,0.023487,0.002437,0.004154,0.002029,0.003056,0.093302,0.057185,0.016235,...,0.002224,0.020908,0.001437,0.065164,0.001618,0.001186,0.051443,3,2.486128,topic_duurzaamheid


### Load userneeds data

In [9]:
ddf_userneeds = dd.read_csv(f'data_dpg_testdata/{DATA_FOLDER_ARTICLES}/reduced_articles_df_fixed_set_userneeds.csv', dtype={'article_id':'str'})
columns_userneeds = ddf_userneeds.columns[ddf_userneeds.columns.str.startswith("userneed_")].values
ddf_userneeds = ddf_userneeds.assign(argmax_userneeds = ddf_userneeds[columns_userneeds].idxmax(axis=1))
ddf_userneeds = ddf_userneeds.set_index('article_id')
ddf_userneeds.head()

,userneed_hou_me_op_de_hoogte,userneed_geef_me_context,userneed_vermaak_me,userneed_raak_me_verbind_me,userneed_help_me,file_name,argmax_userneeds
article_id,,,,,,,
1013291,0.132751,0.059254,0.704135,0.089935,0.013925,2,userneed_vermaak_me
1149251,0.159672,0.108233,0.444364,0.015201,0.272529,3,userneed_vermaak_me
1401797,0.071099,0.054092,0.790514,0.059013,0.025282,3,userneed_vermaak_me
1420717,0.393687,0.159692,0.321118,0.030522,0.094981,3,userneed_hou_me_op_de_hoogte
1505884,0.149060,0.400487,0.137633,0.304317,0.008503,3,userneed_geef_me_context


### Load sensitivity data

In [10]:
ddf_sensitivity = dd.read_csv(f'data_dpg_testdata/{DATA_FOLDER_ARTICLES}/reduced_articles_df_fixed_set_sensitive.csv', dtype={'article_id':'str'})
columns_sensitivity = list(ddf_sensitivity.columns[ddf_sensitivity.columns.str.startswith("sensitive_")].values)
ddf_sensitivity = ddf_sensitivity.assign(argmax_sensitivity = ddf_sensitivity[columns_sensitivity].idxmax(axis=1))
ddf_sensitivity = ddf_sensitivity.assign(is_sensitive = ddf_sensitivity[columns_sensitivity].sum(axis=1))
ddf_sensitivity = ddf_sensitivity.dropna(subset="sensitive_topic_terrorism")
ddf_sensitivity = ddf_sensitivity.assign(article_sensitivity_entropy = ddf_sensitivity[columns_sensitivity].apply(lambda x: entropy(x) if x.sum()!=0 else 0, axis=1, meta='float64'))
ddf_sensitivity = ddf_sensitivity.set_index('article_id')
# ddf_sensitivity = ddf_sensitivity.persist()
ddf_sensitivity.head()

/home/bestname/anaconda3/envs/dpg/lib/python3.10/site-packages/dask/dataframe/core.py:6751: FutureWarning: Meta is not valid, `map_partitions` and `map_overlap` expects output to be a pandas object. Try passing a pandas object as meta or a dict or tuple representing the (name, dtype) of the columns. In the future the meta you passed will not work.
  warnings.warn(


,sensitive_topic_adult,sensitive_topic_arms,sensitive_topic_crime,sensitive_topic_death,sensitive_topic_drugs,sensitive_topic_hate_speech,sensitive_topic_obscenity,sensitive_topic_piracy,sensitive_topic_sensitive,sensitive_topic_social_issue,sensitive_topic_terrorism,sensitive_topic_sensitive_and_social_issue,sensitive_topic_spam_and_harmful_content,file_name,argmax_sensitivity,is_sensitive,article_sensitivity_entropy
article_id,,,,,,,,,,,,,,,,,
1013291,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.341225,0.0,0.341225,0.317549,2,sensitive_topic_social_issue,1.0,1.098047
1149251,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.000000,3,sensitive_topic_adult,0.0,0.000000
1401797,0.0,0.0,0.0,0.132023,0.0,0.0,0.0,0.0,0.0,0.433988,0.0,0.433988,0.000000,3,sensitive_topic_social_issue,1.0,0.991850
1420717,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.500000,0.0,0.500000,0.000000,3,sensitive_topic_social_issue,1.0,0.693147
1505884,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.000000,3,sensitive_topic_adult,0.0,0.000000


# Load samples

In [3]:
# Article data

ddf_articles = dd.read_csv("../data/sample_articles.csv")
#ddf_articles.head(2)

# Select relevant columns
ddf_arts_short = ddf_articles[["title", "num_words", "categories_generated", "keywords_curated"]]
ddf_arts_short.head()

,title,num_words,categories_generated,keywords_curated
0,Peugeot 307 in alle opzichten verbeterd,351,Onderweg,NaN
1,Hier is hij dan: de nieuwe Volvo V70!,0,Onderweg,NaN
2,De Peugeot 207 SW!,170,Onderweg,NaN
3,Meer details nieuwe Opel Agila,361,Onderweg,NaN
4,Oranje meest populair voor Dodge Charger,178,Onderweg,NaN


In [4]:
# Interaction data
ddf_views = dd.read_csv("../data/views_sample.csv", assume_missing = True)
ddf_views['ARTICLE_ID'] = ddf_views['ARTICLE_ID'].astype(int)
ddf_views = ddf_views.drop("GEO_ZIPCODE", axis = 1)
ddf_views = ddf_views.drop("article_id", axis = 1)


In [5]:
ddf_views.head()

,APP_ID,ARTICLE_ID,DERIVED_TSTAMP,GEO_COUNTRY,REFR_URLHOST,QUASI_USER_ID,IS_LOGGED_IN,dt_hour,dt_weekday,dt_dayofmonth,dt_month
0,nu.android,6220992,2022-08-31 11:10:26.133000+00:00,NL,5.0,9ac49e695e84ed528977e918a8b56690,0.0,11.0,2.0,31.0,8.0
1,nu.web,6221051,2022-09-01 18:15:48.771000+00:00,NL,3.0,6cfbc147ff75fafd27f0ff0c507ef326,1.0,18.0,3.0,1.0,9.0
2,nu.ios,6221091,2022-09-01 05:44:20.143000+00:00,NL,5.0,735d4a9f97ba8a359098338320aa8164,0.0,5.0,3.0,1.0,9.0
3,nu.ios,6221108,2022-08-31 18:11:34.921000+00:00,NL,5.0,bbf534b9b2be63b5dc581c8ff9993e6e,0.0,18.0,2.0,31.0,8.0
4,nu.ios,6221144,2022-09-01 05:25:56.571000+00:00,NL,5.0,a2f6212712e17df13778e86841261ade,0.0,5.0,3.0,1.0,9.0


In [6]:
# Userneeds data
ddf_needs = dd.read_csv("../Data/userneeds_sample.csv", sep = ";")
ddf_needs = ddf_needs.drop("file_name", axis = 1)

ddf_needs.head()

,ARTICLE_ID,QUASI_USER_ID,userneed_geef_me_context,userneed_help_me,userneed_hou_me_op_de_hoogte,userneed_raak_me_verbind_me,userneed_vermaak_me
0,6221650,5ca2ac94fb5918645a2b3eb386d15824,"0,11957381","0,044533264","0,12308365","0,08426891","0,6285405"
1,6221652,0319480f035fcf216149dd68cfa7fb10,"0,017739477","0,002837213","0,009671224","0,96193784","0,00781424"
2,6221699,08f41051f4a4c7b2f3fa728176536b04,"0,7628144","0,006565137","0,12116866","0,03433455","0,07511718"
3,6221703,4e66c57b61aa18b7e45e8fa743ef985c,"0,61539996","0,02302798","0,100051075","0,056694284","0,20482665"
4,6221742,13b3a9adbd5244105a270d4d56a46b3a,"0,027910735","0,003595882","0,031106325","0,9220885","0,015298569"


In [7]:
article_ids = ddf_needs["ARTICLE_ID"]
ddf_arts_short["ARTICLE_ID"] = article_ids
ddf_arts_short.head()

,title,num_words,categories_generated,keywords_curated,ARTICLE_ID
0,Peugeot 307 in alle opzichten verbeterd,351,Onderweg,NaN,6221650
1,Hier is hij dan: de nieuwe Volvo V70!,0,Onderweg,NaN,6221652
2,De Peugeot 207 SW!,170,Onderweg,NaN,6221699
3,Meer details nieuwe Opel Agila,361,Onderweg,NaN,6221703
4,Oranje meest populair voor Dodge Charger,178,Onderweg,NaN,6221742


In [8]:
ddf_needs = ddf_needs.drop("ARTICLE_ID", axis = 1)

# Preprocessing

TO DO: 
- merge dataframes (interactions, views, articles) by matching them on column variables they have in common
- map strings to numerical values
- remove unneccesary columns 


### Merge dataframes 

In [9]:
# Merge dataframes 

# Merge the two dataframes on the 'article_id' column
merged_df = dd.merge(ddf_views, ddf_arts_short, on='ARTICLE_ID')
# compute the result
result1 = merged_df.compute()

# Repeat to include the userneeds dataframe
merged_df2 = dd.merge(result1, ddf_needs, on="QUASI_USER_ID")
df = merged_df2.compute()

# From now on, continue to work with this dataframe, it contains all relevant columns
df

,APP_ID,ARTICLE_ID,DERIVED_TSTAMP,GEO_COUNTRY,REFR_URLHOST,QUASI_USER_ID,IS_LOGGED_IN,dt_hour,dt_weekday,dt_dayofmonth,dt_month,title,num_words,categories_generated,keywords_curated,userneed_geef_me_context,userneed_help_me,userneed_hou_me_op_de_hoogte,userneed_raak_me_verbind_me,userneed_vermaak_me
0,nu.android,6221650,2022-09-02 23:53:32.502000+00:00,ES,5.0,5ca2ac94fb5918645a2b3eb386d15824,0.0,23.0,4.0,2.0,9.0,Peugeot 307 in alle opzichten verbeterd,351,Onderweg,NaN,"0,11957381","0,044533264","0,12308365","0,08426891","0,6285405"
1,nu.ios,6221652,2022-09-03 06:49:11.242000+00:00,NL,5.0,0319480f035fcf216149dd68cfa7fb10,0.0,6.0,5.0,3.0,9.0,Hier is hij dan: de nieuwe Volvo V70!,0,Onderweg,NaN,"0,017739477","0,002837213","0,009671224","0,96193784","0,00781424"
2,nu.web,6221699,2022-09-06 14:44:35.209000+00:00,NL,1.0,08f41051f4a4c7b2f3fa728176536b04,1.0,14.0,1.0,6.0,9.0,De Peugeot 207 SW!,170,Onderweg,NaN,"0,7628144","0,006565137","0,12116866","0,03433455","0,07511718"
3,nu.web,6221703,2022-09-03 17:23:47.587000+00:00,NL,3.0,4e66c57b61aa18b7e45e8fa743ef985c,0.0,17.0,5.0,3.0,9.0,Meer details nieuwe Opel Agila,361,Onderweg,NaN,"0,61539996","0,02302798","0,100051075","0,056694284","0,20482665"
4,nu.web,6221742,2022-09-03 14:48:33.425000+00:00,NL,1.0,13b3a9adbd5244105a270d4d56a46b3a,1.0,14.0,5.0,3.0,9.0,Oranje meest populair voor Dodge Charger,178,Onderweg,NaN,"0,027910735","0,003595882","0,031106325","0,9220885","0,015298569"
5,nu.web,6221753,2022-09-03 22:22:15.356000+00:00,NL,1.0,f4e52064fbd141d03c9cf04d317e6e8b,1.0,22.0,5.0,3.0,9.0,Nieuwe Celica in samenwerking met Subaru,206,Onderweg,NaN,"0,07931958","0,46115047","0,037039593","0,01969836","0,40279222"
6,nu.ios,6221776,2022-09-03 17:31:08.891000+00:00,NL,5.0,93ba947a5048dae184389ab632961bc0,0.0,17.0,5.0,3.0,9.0,Waar komt de Volkswagen Up vandaan?,152,Onderweg,NaN,"0,002247357","0,95498496","0,003490555","0,005730557","0,03354662"
7,nu.web,6221793,2022-09-03 19:23:56.470000+00:00,NL,3.0,c57f1e19b40975729ef94addb065b228,1.0,19.0,5.0,3.0,9.0,Dodge Ram gaat groen,134,Onderweg,NaN,"0,20585853","0,17376249","0,022047","0,060941808","0,53739005"
8,nu.web,6221798,2022-09-04 12:40:25.066000+00:00,NL,6.0,7f07a0800cfb1654d61b7373e44d8568,1.0,12.0,6.0,4.0,9.0,Eindelijk: de nieuwe Suzuki Alto,0,Onderweg,NaN,"0,6423039","0,018064383","0,019873463","0,15361337","0,1661448"
9,nu.ios,6221816,2022-09-03 21:37:01.963000+00:00,NL,5.0,1cfb85e7d42fba92ad8d01b05e481253,0.0,21.0,5.0,3.0,9.0,Peugeot 107 volgt zijn broertje,183,Onderweg|Overig,NaN,"0,4451582","0,12436104","0,12181396","0,028075831","0,2805911"


### Make mappings 

In [10]:
def make_mapping(df, variable):
    """
    You can use this function to map variables in a dataframe to a number. Just feed the 
    dataframe and column name into this function. 

    :input df:        Dataframe 
    :input variable:  String, name of the column you want to map to numbers
    :output df :      Dataframe
    
    """
    
    column_list = df[variable]
    
    # Get the unique list values while preserving the input order 
    unique_values = list(OrderedDict.fromkeys(column_list))
    
    # Loop over the unique values and add the mappings to a lookup dictionary
    mapping = {}
    for i, s in enumerate(unique_values):
        mapping[s] = i
        
    # Replace values using the lookup dictionary
    df = df.replace({variable: mapping})
    
    return df

    

In [11]:
# Apply mappings 

df = make_mapping(df, "GEO_COUNTRY")
df = make_mapping(df, "APP_ID")
#df = make_mapping(df, "REFR_MEDIUM")
df = make_mapping(df, "QUASI_USER_ID")

df.head()

,APP_ID,ARTICLE_ID,DERIVED_TSTAMP,GEO_COUNTRY,REFR_URLHOST,QUASI_USER_ID,IS_LOGGED_IN,dt_hour,dt_weekday,dt_dayofmonth,dt_month,title,num_words,categories_generated,keywords_curated,userneed_geef_me_context,userneed_help_me,userneed_hou_me_op_de_hoogte,userneed_raak_me_verbind_me,userneed_vermaak_me
0,0,6221650,2022-09-02 23:53:32.502000+00:00,0,5.0,0,0.0,23.0,4.0,2.0,9.0,Peugeot 307 in alle opzichten verbeterd,351,Onderweg,NaN,"0,11957381","0,044533264","0,12308365","0,08426891","0,6285405"
1,1,6221652,2022-09-03 06:49:11.242000+00:00,1,5.0,1,0.0,6.0,5.0,3.0,9.0,Hier is hij dan: de nieuwe Volvo V70!,0,Onderweg,NaN,"0,017739477","0,002837213","0,009671224","0,96193784","0,00781424"
2,2,6221699,2022-09-06 14:44:35.209000+00:00,1,1.0,2,1.0,14.0,1.0,6.0,9.0,De Peugeot 207 SW!,170,Onderweg,NaN,"0,7628144","0,006565137","0,12116866","0,03433455","0,07511718"
3,2,6221703,2022-09-03 17:23:47.587000+00:00,1,3.0,3,0.0,17.0,5.0,3.0,9.0,Meer details nieuwe Opel Agila,361,Onderweg,NaN,"0,61539996","0,02302798","0,100051075","0,056694284","0,20482665"
4,2,6221742,2022-09-03 14:48:33.425000+00:00,1,1.0,4,1.0,14.0,5.0,3.0,9.0,Oranje meest populair voor Dodge Charger,178,Onderweg,NaN,"0,027910735","0,003595882","0,031106325","0,9220885","0,015298569"


In [83]:
# User needs mapping

#df = ddf_needs

def userneeds_mapping(df):
    '''
    Input: An entire dataframe for now. Has to be converted to input of a row

    Output: Vectors containing the probabilities of 5 userneeds
    '''
    # Define a function that returns a vector of values from multiple columns in a DataFrame
    def get_vector(row):
        return [row["userneed_geef_me_context"], row["userneed_help_me"], row["userneed_hou_me_op_de_hoogte"], row["userneed_raak_me_verbind_me"], row["userneed_vermaak_me"]]

    # Create a new column containing the vector of values from multiple columns using the apply method
    df['vector'] = df.apply(get_vector, axis=1, meta=('vector', 'f8'))

    # Compute the result
    result = df.compute()

    return result.loc[:, 'vector']

In [114]:
df.columns

Index(['APP_ID', 'ARTICLE_ID', 'DERIVED_TSTAMP', 'GEO_COUNTRY', 'REFR_URLHOST',
       'QUASI_USER_ID', 'IS_LOGGED_IN', 'dt_hour', 'dt_weekday',
       'dt_dayofmonth', 'dt_month', 'title', 'num_words',
       'categories_generated', 'keywords_curated', 'userneed_geef_me_context',
       'userneed_help_me', 'userneed_hou_me_op_de_hoogte',
       'userneed_raak_me_verbind_me', 'userneed_vermaak_me'],
      dtype='object')

# Build features

In [12]:
def make_feature_vector(df):
    """
    input: row of a dataframe 
    
    output: a list containing 1 feature vector
    
    """
    
    # Select the columns that can be directly converted into features 
    
    #user_id = df["QUASI_USER_ID"]
    article_id = int(df["ARTICLE_ID"])
    user_id = int(df["QUASI_USER_ID"])
    hour = int(df["dt_hour"])
    weekday = int(df["dt_weekday"])
    
    login = int(df["IS_LOGGED_IN"])
    num_words = int(df["num_words"]) # In the article data, a lot of articles have 0 in this column. That could skew the results
    
    #userneeds = df["userneed_geef_me_context"], df["userneed_help_me"], df["userneed_hou_me_op_de_hoogte"], df['userneed_raak_me_verbind_me'], df['userneed_vermaak_me']
    
    # Make vector 
    #vector = [article_id, hour, weekday, login, num_words, userneeds]
    
    vector = [article_id, user_id, hour, weekday, login, num_words]
    
    return vector
    

In [127]:
# The .apply function performs the make_feature_vector function to all rows in the dataframe
features = df.apply(make_feature_vector, axis = 1)
features = features.tolist()

In [128]:
print(type(features))
print(features)

<class 'list'>
[[6221650, 0, 23, 4, 0, 351], [6221652, 1, 6, 5, 0, 0], [6221699, 2, 14, 1, 1, 170], [6221703, 3, 17, 5, 0, 361], [6221742, 4, 14, 5, 1, 178], [6221753, 5, 22, 5, 1, 206], [6221776, 6, 17, 5, 0, 152], [6221793, 7, 19, 5, 1, 134], [6221798, 8, 12, 6, 1, 0], [6221816, 9, 21, 5, 0, 183]]


# K-Means Clustering

In [75]:
from sklearn.cluster import KMeans, DBSCAN
from sklearn.metrics import silhouette_score, davies_bouldin_score 